In [22]:
#impot dependencies
import pandas as pd
import numpy as np
import json
import ast
from config import api_key, app_id
import requests


In [2]:
all_organizations = pd.read_csv("All_Organizations.csv")
all_organizations_df = pd.DataFrame(all_organizations)
all_organizations_df.head()
all_organizations_df = all_organizations_df.drop_duplicates('ein')

In [3]:
all_organizations_df = all_organizations_df.dropna(subset=["currentRating"])
all_organizations_df.reindex(columns=['charityName', 'cause',  'mailingAddress', 'orgID', 'Category Name', 'Rating', 'irsClassification'])
all_organizations_df.head()


,Unnamed: 0,advisories,category,cause,charityName,charityNavigatorURL,currentRating,donationAddress,ein,irsClassification,mailingAddress,mission,orgID,organization,tagLine,websiteURL
0,0,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'Research and Public Policy',...","{'causeID': 35, 'causeName': 'Non-Medical Scie...",Mount Desert Island Biological Laboratory,https://www.charitynavigator.org/?bay=search.s...,"{'score': 92.61, 'ratingID': 131800, 'publicat...","{'country': None, 'stateOrProvince': 'ME', 'ci...",10202467,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'ME', 'ci...",The MDI Biological Laboratory is a rapidly gro...,5954,{'charityName': 'Mount Desert Island Biologica...,"Connecting Science, Environment, and Health",http://www.mdibl.org/
1,1,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'Community Development', 'cat...","{'causeID': 42, 'causeName': 'United Ways', 'c...",United Way of Eastern Maine,https://www.charitynavigator.org/?bay=search.s...,"{'score': 81.84, 'ratingID': 138738, 'publicat...",NaN,10211478,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'ME', 'ci...","Working with the communities we serve, the Uni...",12517,"{'charityName': 'United Way of Eastern Maine',...",Live united,https://www.unitedwayem.org/
2,2,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'Health', 'categoryID': 5, 'c...","{'causeID': 14, 'causeName': 'Medical Research...",The Jackson Laboratory,https://www.charitynavigator.org/?bay=search.s...,"{'score': 93.16, 'ratingID': 131608, 'publicat...","{'country': None, 'stateOrProvince': 'ME', 'ci...",10211513,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'ME', 'ci...","The Jackson Laboratory, founded in 1929, is an...",3916,"{'charityName': 'The Jackson Laboratory', 'ein...",Leading the search for tomorrow's cures,http://www.jax.org
3,3,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'Arts, Culture, Humanities', ...","{'causeID': 6, 'causeName': 'Libraries, Histor...",Maine Historical Society,https://www.charitynavigator.org/?bay=search.s...,"{'score': 80.31, 'ratingID': 124121, 'publicat...",NaN,10211530,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'ME', 'ci...","Founded in 1822, the Maine Historical Society ...",15533,"{'charityName': 'Maine Historical Society', 'e...","Preserving history, engaging minds, connecting...",http://www.mainehistory.org/
4,4,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'Human Services', 'categoryID...","{'causeID': 16, 'causeName': 'Youth Developmen...",Boys & Girls Clubs of Southern Maine,https://www.charitynavigator.org/?bay=search.s...,"{'score': 91.54, 'ratingID': 135033, 'publicat...",NaN,10211543,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'ME', 'ci...","For more than 100 years, Boys & Girls Clubs of...",15222,{'charityName': 'Boys & Girls Clubs of Souther...,Great Futures Start Here,http://www.bgcmaine.org/home


In [6]:
def getCategoryName(dict_string):
    return ast.literal_eval(dict_string)["categoryName"]
def getCurrentRating(dict_string):
    dictionary = ast.literal_eval(dict_string)
    return dictionary["score"]
def getMailingAddress(dict_string):
    dictionary = ast.literal_eval(dict_string)
    return dictionary['stateOrProvince']
def getIncomeAmount(dict_string):
    dictionary = ast.literal_eval(dict_string)
    return dictionary['incomeAmount']
def getRatingID(dict_string):
    dictionary = ast.literal_eval(dict_string)
    return dictionary["ratingID"]

all_organizations_df["Category Name"] = all_organizations_df["category"].apply(getCategoryName)
all_organizations_df["Rating"] = all_organizations_df["currentRating"].apply(getCurrentRating)
all_organizations_df["State"] = all_organizations_df['mailingAddress'].apply(getMailingAddress)
all_organizations_df["Contributions"] = all_organizations_df['irsClassification'].apply(getIncomeAmount)
#

In [33]:
rating_objects = {}

In [59]:
default_rating_params = {
    "app_id": app_id,
    "app_key": api_key
}

def createRatingURL(ein, rating_id):
    base_organization_url = "https://api.data.charitynavigator.org/v2/Organizations"
    query_params = [f"{param}={value}" for (param, value) in default_rating_params.items()]
    query_string = "&".join(query_params)
    return f"{base_organization_url}/{ein}/Ratings/{rating_id}?{query_string}"
    
def getCurrentRatingObject(dataframe_row):
    rating_id = getRatingID(dataframe_row["currentRating"])
    ein = dataframe_row["ein"]
    url = createRatingURL(ein, rating_id)
    rating_object = requests.get(url).json()
    organization_name = dataframe_row["charityName"]
    rating_objects[organization_name] = rating_object
    print(f"finished: {organization_name}")
    return rating_object

In [53]:
rating_objects = {
}

In [67]:
recorded_organization_ratings = rating_objects.keys()
missing_ratings = all_organizations_df[~all_organizations_df["charityName"].isin(recorded_organization_ratings)]
recently_retrieved_ratings = missing_ratings.apply(getCurrentRatingObject, axis="columns")


In [ ]:
len(list(rating_objects.items()))
ratings = rating_objects.values()
ratings = list(ratings)
ratings_df = pd.DataFrame(ratings)
ratings_df.head()

In [6]:
all_orgs_trim_df = all_organizations_df[['charityName', 'ein', 'mission', 'cause', "Category Name", "Rating", "State", "Contributions"]]
all_orgs_trim_df = all_orgs_trim_df.rename(columns={'charityName': 'Charity', 'orgID': 'ID_#', 'mission': 'Mission', 'cause': 'Cause'})
all_orgs_trim_df.head()

,Charity,ein,Mission,Cause,Category Name,Rating,State,Contributions
0,Mount Desert Island Biological Laboratory,10202467,The MDI Biological Laboratory is a rapidly gro...,"{'causeID': 35, 'causeName': 'Non-Medical Scie...",Research and Public Policy,92.61,ME,12924245.0
1,United Way of Eastern Maine,10211478,"Working with the communities we serve, the Uni...","{'causeID': 42, 'causeName': 'United Ways', 'c...",Community Development,81.84,ME,2438155.0
2,The Jackson Laboratory,10211513,"The Jackson Laboratory, founded in 1929, is an...","{'causeID': 14, 'causeName': 'Medical Research...",Health,93.16,ME,331241336.0
3,Maine Historical Society,10211530,"Founded in 1822, the Maine Historical Society ...","{'causeID': 6, 'causeName': 'Libraries, Histor...","Arts, Culture, Humanities",80.31,ME,2219692.0
4,Boys & Girls Clubs of Southern Maine,10211543,"For more than 100 years, Boys & Girls Clubs of...","{'causeID': 16, 'causeName': 'Youth Developmen...",Human Services,91.54,ME,3859985.0


In [18]:
# Output to a csvfile for the team to work with
all_orgs_trim_df.to_csv(r'cleaned_data.csv')